In [1]:
import librosa
import numpy as np
import torch

from src.constants import SAMPLE_RATE, WINDOW_LENGTH

HOP_LENGTH = 20
HOP_LENGTH = int(HOP_LENGTH / 1000 * SAMPLE_RATE)
SAMPLE_RATE = 16000

audio, _ = librosa.load("input/j3ko_3s.wav", sr=SAMPLE_RATE)
audio_l = len(audio)
audio = np.pad(audio, WINDOW_LENGTH // 2, mode="constant")

audio = torch.from_numpy(audio).float()
audio_steps = audio_l // HOP_LENGTH + 1

seq_len = 2.55
seq_len = int(seq_len * SAMPLE_RATE)

data = []

n_steps = seq_len // HOP_LENGTH + 1
for i in range(audio_l // seq_len):
    begin_t = i * seq_len
    end_t = begin_t + seq_len + WINDOW_LENGTH
    begin_step = begin_t // HOP_LENGTH
    end_step = begin_step + n_steps
    data.append(
        audio[begin_t:end_t],
    )
data.append(
    audio[-seq_len - WINDOW_LENGTH :],
)



In [25]:
from src.model import E2E


model = torch.load('runs/Pitch_FL6_0/model.pt')
model.eval()
model = model.cuda()

In [26]:
data = torch.stack(data).cuda()

In [27]:
data.shape

torch.Size([2, 42848])

In [20]:
with torch.no_grad():
    result = model(data)


In [21]:
result = result[1]

In [22]:
result.shape

torch.Size([2, 128, 360])

In [23]:
from src.utils import to_local_average_cents


cents_pred = to_local_average_cents(result.view(-1,360).cpu().numpy(),None, 0 )

In [7]:
freq_pred = np.array(
            [
                10 * (2 ** (cent_pred / 1200)) if cent_pred else 0
                for cent_pred in cents_pred
            ]
        )
freq = np.array(
    [10 * (2 ** (cent / 1200)) if cent else 0 for cent in cents_pred]
)

In [8]:
import pandas as pd

In [39]:
df = pd.DataFrame({"frequency": freq})

In [40]:
df.to_csv('f0.csv', index=False)